In [5]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# # Charger le dataframe
# try:
#     df_sample = pd.read_csv('new_merged_file - new_merged_file.csv')
# except Exception as e:
#     print(f"Erreur lors du chargement du fichier CSV : {e}")
#     raise

# # Fonction de recommandation basée sur un thème recherché
# def recommander_par_theme(df, mot_cle, nombre_recommandations=5):
#     mot_cle = mot_cle.lower()

#     # 1. Recherche du mot clé dans la colonne 'genres'
#     films_genre = df[df['genres'].str.contains(mot_cle, case=False, na=False)]

#     if not films_genre.empty:
#         # Si des films sont trouvés dans 'genres', les retourner
#         return films_genre[['title', 'genres']].head(nombre_recommandations)

#     # 2. Si le thème n'est pas trouvé dans 'genres', on cherche dans 'tags'
#     films_tags = df[df['tag'].str.contains(mot_cle, case=False, na=False)]

#     if films_tags.empty:
#         print(f"Aucun film trouvé avec le thème '{mot_cle}' dans les genres ou les tags.")
#         return []

#     # Créer une matrice TF-IDF pour les tags
#     tfidf = TfidfVectorizer(stop_words='english')

#     # Échantillonner pour réduire la taille si nécessaire
#     if len(df) > 10000:  # Par exemple, si plus de 10 000 films
#         df = df.sample(n=10000)  # Limiter à 10 000 films

#     # Remplir les valeurs manquantes dans la colonne 'tag'
#     df['tag'] = df['tag'].fillna('')
#     tfidf_matrix = tfidf.fit_transform(df['tag'])

#     # Calculer la similarité cosinus entre les films
#     cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

#     # Obtenir l'index du premier film trouvé correspondant au tag
#     idx = films_tags.index[0]

#     # Calculer les scores de similarité pour ce film
#     sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:nombre_recommandations+1]  # Exclure le film de base

#     # Récupérer les indices des films les plus similaires
#     film_indices = [i[0] for i in sim_scores]

#     # Retourner les titres et genres des films recommandés
#     recommandations = df[['title', 'genres']].iloc[film_indices]
#     return recommandations

# # Exemple d'utilisation
# mot_cle = 'women'  # Thème à rechercher
# nombre_recommandations = 5  # Nombre de films à recommander

# film_recommandes = recommander_par_theme(df_sample, mot_cle, nombre_recommandations)

# # Afficher les résultats
# if not film_recommandes.empty:
#     print("Films recommandés :")
#     print(film_recommandes)
# else:
#     print("Aucune recommandation disponible.")



Films recommandés :
                                     title                        genres
314721  Irreversible (Irréversible) (2002)  Crime|Drama|Mystery|Thriller
796534                American Made (2017)                Crime|Thriller
598337         The Man from Nowhere (2010)         Action|Crime|Thriller
656632                Europa Report (2013)               Sci-Fi|Thriller
606292   Birdemic: Shock and Terror (2010)              Romance|Thriller


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Charger le dataframe en utilisant un échantillon si nécessaire
try:
    df_sample = pd.read_csv('new_merged_file - new_merged_file.csv')

    # Échantillonner pour réduire la taille si nécessaire
    if len(df_sample) > 10000:  # Limiter à 10 000 films
        df_sample = df_sample.sample(n=10000, random_state=1).reset_index(drop=True)

except Exception as e:
    print(f"Erreur lors du chargement du fichier CSV : {e}")
    raise

# Fonction de recommandation intelligente basée sur un thème recherché
def recommander_meilleur_film(df, mot_cle):
    mot_cle = mot_cle.lower()

    # 1. Recherche du mot clé dans la colonne 'genres'
    films_genre = df[df['genres'].str.contains(mot_cle, case=False, na=False)]

    if not films_genre.empty:
        # Si des films sont trouvés dans 'genres', choisir le meilleur film
        return films_genre[['title', 'genres']].iloc[0]

    # 2. Si le thème n'est pas trouvé dans 'genres', on cherche dans 'tags'
    films_tags = df[df['tag'].str.contains(mot_cle, case=False, na=False)]

    if films_tags.empty:
        print(f"Aucun film trouvé avec le thème '{mot_cle}' dans les genres ou les tags.")
        return None

    # Créer une matrice TF-IDF pour les genres
    tfidf_genres = TfidfVectorizer(stop_words='english')
    tfidf_matrix_genres = tfidf_genres.fit_transform(df['genres'].fillna(''))  # Remplir les valeurs manquantes

    # Créer une matrice TF-IDF pour les tags
    tfidf_tags = TfidfVectorizer(stop_words='english')
    df['tag'] = df['tag'].fillna('')  # Remplir les valeurs manquantes
    tfidf_matrix_tags = tfidf_tags.fit_transform(df['tag'])

    # Calculer la similarité cosinus pour les genres et les tags
    cosine_sim_genres = cosine_similarity(tfidf_matrix_genres, tfidf_matrix_genres)
    cosine_sim_tags = cosine_similarity(tfidf_matrix_tags, tfidf_matrix_tags)

    # Obtenir l'index du premier film trouvé correspondant au tag
    idx = films_tags.index[0]

    # Calculer les scores de similarité pour ce film dans les deux matrices
    sim_scores_genres = list(enumerate(cosine_sim_genres[idx]))
    sim_scores_tags = list(enumerate(cosine_sim_tags[idx]))

    # Combiner les scores de similarité
    combined_scores = {}
    for i in range(len(df)):
        combined_scores[i] = (sim_scores_genres[i][1] + sim_scores_tags[i][1]) / 2  # Moyenne des deux scores

    # Trier les films en fonction des scores combinés
    sorted_scores = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)

    # Retourner le film avec le score le plus élevé
    best_film_idx = sorted_scores[0][0]
    return df[['title', 'genres']].iloc[best_film_idx]

# Exemple d'utilisation
mot_cle = 'women'  # Thème à rechercher
film_recommande = recommander_meilleur_film(df_sample, mot_cle)

# Afficher les résultats
if film_recommande is not None:
    print("Film recommandé :")
    print(film_recommande)
else:
    print("Aucune recommandation disponible.")


Film recommandé :
title     Kill Bill: Vol. 1 (2003)
genres       Action|Crime|Thriller
Name: 261, dtype: object


In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Charger le dataframe en utilisant un échantillon si nécessaire
try:
    df_sample = pd.read_csv('new_merged_file - new_merged_file.csv')

    # Échantillonner pour réduire la taille si nécessaire
    if len(df_sample) > 10000:  # Limiter à 10 000 films
        df_sample = df_sample.sample(n=10000, random_state=1).reset_index(drop=True)

except Exception as e:
    print(f"Erreur lors du chargement du fichier CSV : {e}")
    raise

# Fonction de recommandation intelligente basée sur un thème recherché
def recommander_meilleurs_films(df, mot_cle, n_recommandations=5):
    mot_cle = mot_cle.lower()

    # 1. Recherche du mot clé dans la colonne 'genres'
    films_genre = df[df['genres'].str.contains(mot_cle, case=False, na=False)]

    if not films_genre.empty:
        # Si des films sont trouvés dans 'genres', choisir les meilleurs films
        return films_genre[['title', 'genres']].head(n_recommandations)

    # 2. Si le thème n'est pas trouvé dans 'genres', on cherche dans 'tags'
    films_tags = df[df['tag'].str.contains(mot_cle, case=False, na=False)]

    if films_tags.empty:
        print(f"Aucun film trouvé avec le thème '{mot_cle}' dans les genres ou les tags.")
        return None

    # Créer une matrice TF-IDF pour les genres
    tfidf_genres = TfidfVectorizer(stop_words='english')
    tfidf_matrix_genres = tfidf_genres.fit_transform(df['genres'].fillna(''))  # Remplir les valeurs manquantes

    # Créer une matrice TF-IDF pour les tags
    tfidf_tags = TfidfVectorizer(stop_words='english')
    df['tag'] = df['tag'].fillna('')  # Remplir les valeurs manquantes
    tfidf_matrix_tags = tfidf_tags.fit_transform(df['tag'])

    # Calculer la similarité cosinus pour les genres et les tags
    cosine_sim_genres = cosine_similarity(tfidf_matrix_genres, tfidf_matrix_genres)
    cosine_sim_tags = cosine_similarity(tfidf_matrix_tags, tfidf_matrix_tags)

    # Obtenir l'index du premier film trouvé correspondant au tag
    idx = films_tags.index[0]

    # Calculer les scores de similarité pour ce film dans les deux matrices
    sim_scores_genres = list(enumerate(cosine_sim_genres[idx]))
    sim_scores_tags = list(enumerate(cosine_sim_tags[idx]))

    # Combiner les scores de similarité
    combined_scores = {}
    for i in range(len(df)):
        combined_scores[i] = (sim_scores_genres[i][1] + sim_scores_tags[i][1]) / 2  # Moyenne des deux scores

    # Trier les films en fonction des scores combinés
    sorted_scores = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)

    # Obtenir les indices des meilleurs films
    top_indices = [sorted_scores[i][0] for i in range(min(n_recommandations, len(sorted_scores)))]

    return df[['title', 'genres']].iloc[top_indices]

# Exemple d'utilisation
mot_cle = 'animal'  # Thème à rechercher
films_recommandes = recommander_meilleurs_films(df_sample, mot_cle, n_recommandations=5)

# Afficher les résultats
if films_recommandes is not None:
    print("Films recommandés :")
    print(films_recommandes)
else:
    print("Aucune recommandation disponible.")


Films recommandés :
                                   title  \
1230                   Life of Pi (2012)   
7025                      Jumanji (1995)   
8187                      Jumanji (1995)   
8570  Madagascar: Escape 2 Africa (2008)   
2776                Kung Fu Panda (2008)   

                                               genres  
1230                             Adventure|Drama|IMAX  
7025                       Adventure|Children|Fantasy  
8187                       Adventure|Children|Fantasy  
8570  Action|Adventure|Animation|Children|Comedy|IMAX  
2776            Action|Animation|Children|Comedy|IMAX  
